In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

# device = "cuda:0"

In [2]:
# import sys

# # Get the current directory

# # Add the current directory to the Python path

# sys.path.append('.')

In [3]:
import torch
import torch.onnx
import torch.nn as nn
import torch.nn.init as init
import torch.utils.model_zoo as model_zoo
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.models import build_network
from pcdet.datasets import build_dataloader
from pcdet.utils import common_utils
import os
import numpy as np
import torch
# # import onnx
import onnxruntime as ort
import torch.nn as nn

# from typing import Sequence, NamedTuple



In [4]:
# cfg_file = "./onnx_config.yaml"
cfg_file = "./deploy/pillar/config.yaml"
cfg_from_yaml_file(cfg_file, cfg)
if os.path.exists('./deploy_pillar_sfaw_3d_r')==False:
    os.mkdir('./deploy/deploy_pillar_sfaw_3d_r')
log_file = './deploy/deploy_pillar_sfaw_3d_r/log_trt.log'
logger = common_utils.create_logger(log_file, rank=0)
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=1,
    dist=False, workers=8, logger=logger, training=False
)

FileExistsError: [Errno 17] File exists: './deploy/deploy_pillar_sfaw_3d_r'

In [ ]:
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)
# cfg

In [ ]:
ckpt = "./pillar/model.pth"
model.load_params_from_file(filename=ckpt, logger=logger, to_cpu=False, pre_trained_path=None)
model.eval()
model.cuda()

In [ ]:
# batch_dict = torch.load("./batch_dict.pth", map_location="cuda")
# inputs = batch_dict
# inputs

In [ ]:
batch_dict = torch.load("/mnt/nas2/users/eslim/onnx/sfaw/input_dict.pth", map_location="cuda")

inputs = batch_dict
inputs

In [ ]:
inputs_vfe = model.vfe(inputs)
inputs_vfe


In [ ]:
# data_dict = {}
input_dict = {}
input_dict['batch_dict']={}
input_dict['batch_dict']['voxel_features'] = inputs_vfe['voxel_features']
input_dict['batch_dict']['use_lead_xyz'] = inputs_vfe['use_lead_xyz']
input_dict['batch_dict']['batch_size'] = torch.tensor(inputs_vfe['batch_size']).to(device)
input_dict['batch_dict']['pillar_features'] = inputs_vfe['pillar_features']
input_dict['batch_dict']['voxel_coords'] = inputs_vfe['voxel_coords']
# input_dict['batch_dict']['voxel_coords'] = inputs_vfe['voxel_coords']


# data_dict.update(inputs_bev)
# data_dict

In [ ]:
with torch.no_grad():
    DSVT_3d_Backbone = model.backbone_3d
    # dsvtblocks_list = DSVT_Backbone.stage_0
    # layer_norms_list = DSVT_Backbone.residual_norm_stage_0
    # inputs = model.vfe(inputs)
    # voxel_info = DSVT_Backbone.input_layer(inputs)
    # set_voxel_inds_list = [[voxel_info[f'set_voxel_inds_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
    # set_voxel_masks_list = [[voxel_info[f'set_voxel_mask_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
    # pos_embed_list = [[[voxel_info[f'pos_embed_stage{s}_block{b}_shift{i}'] for i in range(2)] for b in range(4)] for s in range(1)]



In [ ]:
input_names = [
    'voxel_features',
    'use_lead_xyz',
    'batch_size',
    'pillar_features',
    'voxel_coords',
]
output_names = ["output",]

In [ ]:
base_name = "./deploy_pillar_sfaw_3d_r/dsvt_3d"
ts_path = f"{base_name}.ts"
onnx_path = f"{base_name}.onnx"

# allptransblocktrt = AllDSVTBlocksTRT(dsvtblocks_list, layer_norms_list).eval().cuda()

torch.onnx.export(
    DSVT_3d_Backbone,
    # alldsvtblockstrt_inputs,
    input_dict,
    onnx_path, 
    input_names=input_names,
    output_names=output_names, 
    # dynamic_axes=dynamic_axes,
    opset_version=14,
)
# https://github.com/Haiyang-W/DSVT/issues/60

In [ ]:
# # test onnx
# ort_session = ort.InferenceSession(onnx_path)
# def to_numpy(tensor):
#     return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# # compute ONNX Runtime output prediction
# ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(pillar_features),
#               ort_session.get_inputs()[1].name: to_numpy(set_voxel_inds_list[0][0]),
#               ort_session.get_inputs()[2].name: to_numpy(set_voxel_inds_list[0][1]),
#               ort_session.get_inputs()[3].name: to_numpy(set_voxel_masks_list[0][0]),
#               ort_session.get_inputs()[4].name: to_numpy(set_voxel_masks_list[0][1]),
#               ort_session.get_inputs()[5].name: to_numpy(torch.stack([torch.stack(v, dim=0) for v in pos_embed_list[0]], dim=0)),}
# ort_outs = ort_session.run(None, ort_inputs)
# ort_outs[0].shape
